<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/video_archeology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [2]:

import geemap
import cv2
import os
import matplotlib.pyplot as plt


In [5]:
!pip install geedim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.2 MB/s eta 0:00:00


In [15]:
import numpy as np
from PIL import Image
aoi = ee.Geometry.BBox(-74.1, -10.5, -73.5, -10.0)  # Amazon basin area (example)
start = '2022-07-01'
end = '2022-07-31'
# NDVI from Sentinel-2
aoi = ee.Geometry.BBox(-74.1, -10.5, -73.5, -10.0)
start, end = '2022-07-01', '2022-07-31'

s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi).filterDate(start, end).median()
ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, end).select('VV').median().rename('Radar')
srtm = ee.Image('USGS/SRTMGL1_003').clip(aoi)
lst = ee.ImageCollection('MODIS/061/MOD11A2').filterBounds(aoi).filterDate(start, end).mean().select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('LST')
#smap = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture_v3').filterBounds(aoi).filterDate(start, end).select('ssm').mean().rename('Soil_Moisture')
smap = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture') \
    .filterBounds(aoi).filterDate(start, end) \
    .select('ssm').mean().rename('Soil_Moisture')

# For each layer, explicitly select only one band before visualization
layers = {
    'NDVI': ndvi.select(0).visualize(min=0, max=1, palette=['white', 'green']),
    'Radar': s1.select(0).visualize(min=-20, max=0, palette=['white', 'black']),
    'Elevation': srtm.select(0).visualize(min=0, max=3000, palette=['#00FFFF', '#000080']),
    'LST': lst.select(0).visualize(min=20, max=45, palette=['blue', 'green', 'red']),
    'Soil_Moisture': smap.select(0).visualize(min=0, max=0.4, palette=['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']),
}

frames = []
vis_params = {
    'NDVI': {'min': 0, 'max': 1, 'palette': ['white', 'green']},
    'Radar': {'min': -20, 'max': 0, 'palette': ['white', 'black']},
    'Elevation': {'min': 0, 'max': 3000, 'palette': ['#00FFFF', '#000080']},
    'LST': {'min': 20, 'max': 45, 'palette': ['blue', 'green', 'red']},
    'Soil_Moisture': {'min': 0, 'max': 0.4, 'palette': ['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']}
}

# Define export projection explicitly
scale = 30  # meters
crs = 'EPSG:4326'

for name, image in layers.items():
    #vis_image = image.visualize(**vis_params[name])
    array = geemap.ee_to_numpy(image, region=aoi, scale=scale)#, crs=crs)#vis_image, region=aoi, scale=scale)#, crs=crs)
    array = np.squeeze(array)

    # Convert to 8-bit RGB and save
    array = np.clip(array, 0, 255).astype(np.uint8)
    image_rgb = Image.fromarray(array)
    image_rgb.save(f"{name}.png")

    # Add to video frames
    frame = cv2.imread(f"{name}.png")
    frames.append(frame)


In [16]:
height, width, _ = frames[0].shape
video = cv2.VideoWriter('archaeology_layers.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

for frame in frames:
    video.write(frame)

video.release()
